<span style="color:black;font-weight:700;font-size:32px"> 
               Построение моделей машинного обучения для фермерского хозяйства "Вольный луг" 
</span>

<span style="color:black;font-weight:700;font-size:22px"> 
              Описание данных 
</span>

Файл ferma_main.csv содержит данные о стаде фермера на текущий момент. Описание таблицы:\

*Параметры корма коровы*\
ЭКЕ (Энергетическая кормовая единица) — измерение питательности корма коровы;\
Сырой протеин — содержание сырого протеина в корме, в граммах;\
СПО (Сахаро-протеиновое соотношение) — отношение сахара к протеину в корме коровы.\

*Где паслась корова*\
Тип пастбища — ландшафт лугов, на которых паслась корова.\
Особенности коровы\
id — номер коровы;\
Порода — порода коровы;\
Возраст — бинарный признак (менее_2_лет, более_2_лет);\
Имя Папы — порода папы коровы.\

*Характеристики молока*:\
Жирность — содержание жиров в молоке, в процентах;\
Белок — содержание белков в молоке, в процентах;\
Вкус молока — оценка вкуса по личным критериям фермера;\
Удой — масса молока, которую корова даёт в год, в килограммах.\

Файл ferma_dad.csv хранит имя папы каждой коровы в стаде фермера.\

Файл cow_buy.csv — это данные о коровах «ЭкоФермы», которых фермер хочет изучить перед покупкой. Структура таблицы почти аналогична ferma_main.csv, кроме нескольких особенностей:\

Белок и Жирность молока указаны на момент покупки — когда «ЭкоФерма» давала им свой корм.

Параметры кормления ЭКЕ, Сырой протеин, СПО отсутствуют. Недавно технологи заказчика пересмотрели параметры корма и планируют их внедрить после расширения стада. Именно новые параметры нужно использовать, когда вы будете применять предсказательные модели к бурёнкам «ЭкоФермы». 

По новой схеме фермер планирует увеличить средние значения ЭКЕ, Сырой протеин, СПО на 5%.
Удой и Вкус молока неизвестны — именно их и предстоит предсказать!

<span style="color:black;font-weight:700;font-size:22px"> 
              План работы
</span>

**1. Загрузка и изучение данных**- изучение данных и основных признаков.\
**2. Предобработка данных** - обработка пропусков, дубликатов(явных, неявных), преобразование типов, обработка выбросов и фильтрация данных.\
**3. Ислледовательский анализ данных**- статистический анализ всех признаков, построение графиков по каждому признаку,вывод о данных\
**4. Коррелляционный анализ**- изучение взаимосвязи между целевым и входныи признаком, визуализация коррелляций, выводы о взаимосвязи.\ 
**5.Обучение модели линейной регрессии**- 
1) Построение модели линейной регресии с данными из датасета ferma_main.csvс целевым признаком Удой.
2) Построение модели линейной регрессии с учётом преобразования признаков, нелинейно связанных с целевым признаком.
3) Построение модели линейной регрессии с учётом нового признака из файла ferma_dad.csv. Учтите преобразование нелинейности в пункте 2.
4) Сравнение качества трёх моделей линейной регрессии, сопоставьте их R² и остатки.
5) Рассчитайте точность каждой модели по трём метрикам: MSE, MAE, RMSE. Используйте функции для расчёта метрики.
    По метрикам определите, какая из моделей точнее.
    Определите доверительный интервал прогноза лучшей модели.

**6. Обучение модели логистической регрессии**- \
**7. Выводы**

# Загрузка и изучение данных

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
try:
    ferma_df = pd.read_csv('/datasets/ferma_main.csv',sep=';')
except:
    ferma_df = pd.read_csv('https://code.s3.yandex.net/datasets/ferma_main.csv', sep=';')

In [17]:
try:
    ferma_dad_df = pd.read_csv('/datasets/ferma_dad.csv',sep=';')
except:
    ferma_dad_df = pd.read_csv('https://code.s3.yandex.net/datasets/ferma_dad.csv',sep=';')

In [25]:
try:
    cow_buy_df = pd.read_csv('/datasets/cow_buy.csv',sep=';')
except:
    cow_buy_df = pd.read_csv('https://code.s3.yandex.net/datasets/cow_buy.csv',sep=';')

In [26]:
ferma_df.head(10)

,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
0,1,5863,"14,2",1743,"0,89",Вис Бик Айдиал,Равнинное,Айдиал,"3,58","3,076",вкусно,более_2_лет
1,2,5529,"12,8",2138,"0,89",Вис Бик Айдиал,Равнинные,Соверин,"3,54","3,079",вкусно,менее_2_лет
2,3,5810,14,1854,"0,885",РефлешнСоверинг,Холмистое,Соверин,"3,59","3,074",не вкусно,более_2_лет
3,4,5895,"12,4",2012,"0,885",РефлешнСоверинг,Холмистое,Айдиал,"3,4","3,075",не вкусно,более_2_лет
4,5,5302,"12,8",1675,"0,885",Вис Бик Айдиал,Равнинные,Соверин,"3,73","3,073",вкусно,менее_2_лет
5,6,5254,"12,7",1806,"0,885",РефлешнСоверинг,Холмистое,Соверин,"3,13","3,078",вкусно,менее_2_лет
6,7,5910,"13,2",1792,"0,885",Вис Бик Айдиал,Равнинные,Соверин,"3,48","3,077",вкусно,более_2_лет
7,8,6179,"14,7",1806,"0,93",Вис Бик Айдиал,Равнинные,Айдиал,"3,55","3,076",вкусно,более_2_лет
8,9,6783,"14,2",1918,"0,94",РефлешнСоверинг,Холмистое,Айдиал,"3,71","3,077",вкусно,более_2_лет
9,10,5339,"13,4",1791,"0,895",Вис Бик Айдиал,Холмистое,Соверин,"3,13","3,075",вкусно,менее_2_лет


In [27]:
ferma_df.shape


(634, 12)

In [28]:
ferma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   id                                     634 non-null    int64 
 1   Удой, кг                               634 non-null    int64 
 2   ЭКЕ (Энергетическая кормовая единица)  634 non-null    object
 3   Сырой протеин, г                       634 non-null    int64 
 4   СПО (Сахаро-протеиновое соотношение)   634 non-null    object
 5   Порода                                 634 non-null    object
 6   Тип пастбища                           634 non-null    object
 7   порода папы_быка                       634 non-null    object
 8   Жирность,%                             634 non-null    object
 9   Белок,%                                634 non-null    object
 10  Вкус молока                            634 non-null    object
 11  Возраст            

In [29]:
ferma_dad_df.head(10)

,id,Имя Папы
0,1,Буйный
1,2,Соловчик
2,3,Барин
3,4,Буйный
4,5,Барин
5,6,Барин
6,7,Соловчик
7,8,Геркулес
8,9,Буйный
9,10,Барин


In [30]:
ferma_dad_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        629 non-null    int64 
 1   Имя Папы  629 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB


In [31]:
cow_buy_df.head(10)

,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%"
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,"3,58","3,076"
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,"3,54","3,081"
2,РефлешнСоверинг,равнинное,Соверин,Барин,"3,59","3,074"
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,"3,4","3,061"
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,"3,64","3,074"
5,РефлешнСоверинг,равнинное,Соверин,Геркулес,"3,63","3,053"
6,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,"3,58","3,076"
7,Вис Бик Айдиал,равнинное,Соверин,Буйный,"3,57","3,079"
8,РефлешнСоверинг,равнинное,Соверин,Барин,"3,59","3,074"
9,РефлешнСоверинг,холмистое,Айдиал,Буйный,"3,4","3,079"


In [32]:
cow_buy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Порода                   20 non-null     object
 1   Тип пастбища             20 non-null     object
 2   порода папы_быка         20 non-null     object
 3   Имя_папы                 20 non-null     object
 4   Текущая_жирность,%       20 non-null     object
 5   Текущий_уровень_белок,%  20 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB
